In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [2]:
load_dotenv(override=True)

True

In [3]:
file = os.getenv("FILE")
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [4]:
loader = PyPDFLoader(file)
files = loader.load()

# Recursive TextSplit

In [5]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=30,
    length_function=len,
    separators= ["\n\n", "\n", ".", " "]
)

In [6]:
documents = recur_split.split_documents(files)

# Embbiding (OpenAI)

In [7]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [8]:
mongo_url = os.getenv("MONGO_URL")
mongo_db = os.getenv("MONGO_DB")
mongo_collection = os.getenv("MONGO_COLLECTION")

In [9]:
client = MongoClient(mongo_url)
db = client[mongo_db]
collection = db[mongo_collection]

In [10]:
vectordb = MongoDBAtlasVectorSearch.from_documents( 
    documents=documents, 
    embedding= embeddings, 
    collection=collection,
    index_name="vector_index"
)

In [11]:
retriever = vectordb.as_retriever(search_type='similarity', search_kwargs = {"k": 3})

# LLM (OpenAI)

In [12]:
llm = ChatOpenAI(model="gpt-4o", api_key=api_key, temperature=0)

# Prompt

In [13]:
prompt = PromptTemplate.from_template(
"""
Você é um assistente de extração de dados educacionais.  
Sua tarefa é localizar **exatamente** os Códigos e Descrições **que já estão presentes no contexto abaixo**.  
**Você NÃO deve criar, reescrever, resumir ou modificar nenhuma informação.**

{context}

REGRAS IMPORTANTES:
- Você **NÃO** pode inventar nenhum conteúdo que não esteja literalmente no contexto acima.
- Você deve retornar **apenas os códigos e descrições exatos e completos** que aparecem no contexto.
- Sempre traga o Codigo e Descricao mais proximos dos dados fornecidos"
- **Não corte partes das descrições.**
- **Não relacione o conteúdo com códigos errados.**

FORMATO OBRIGATÓRIO DE SAÍDA (copiar literalmente do contexto):
Código: <código exato>  
Descrição: <descrição completa>

---

Pergunta do usuário:  
{question}
"""
)

# Chain

In [14]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [21]:
question = "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Uso para calcular juros>."

# Answer

In [22]:
chat_chain.invoke({"query": question})

{'query': '**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Uso para calcular juros>.',
 'result': 'Código: EF09MA06  \nDescrição: Compreender as funções como relações de dependência unívoca entre duas variáveis e suas representações numérica, algébrica e gráfica e utilizar esse conceito para analisar situações que envolvam relações funcionais entre duas variáveis.',
 'source_documents': [Document(id='689e7f1a15c65a06b4a962d7', metadata={'_id': '689e7f1a15c65a06b4a962d7', 'embedding': [-0.0059752133674919605, 0.0026923103723675013, 0.007562224753201008, -0.014134629629552364, -0.014372186735272408, 0.006394237279891968, -0.026962697505950928, -0.016655370593070984, -0.014609742909669876, -0.023148590698838234, 0.018344664946198463, 0.0031080348417162895, -0.003543555736541748, -0.004035166464745998, -0.026170840486884117, 0.006206171587109566, 0.014900090172886848, -0.01433259341865778, 0.013151408173143864, -0.00745664